In [ ]:
# Download data from Github
! git clone https://github.com/NetsecExplained/Machine-Learning-for-Security-Analysts.git

# Install dependencies
! pip install nltk sklearn pandas matplotlib seaborn
data_dir = "Machine-Learning-for-Security-Analysts/Malicious URLs.csv"

Cloning into 'Machine-Learning-for-Security-Analysts'...
remote: Enumerating objects: 4197, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 4197 (delta 4), reused 1 (delta 0), pack-reused 4186
Receiving objects: 100% (4197/4197), 14.84 MiB | 23.74 MiB/s, done.
Resolving deltas: 100% (64/64), done.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
#importing all dependencies
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import re

%matplotlib inline

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# Import Scikit-learn models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report

print('now our libraries has been installed\n')


now our libraries has been installed



In [ ]:
print('loading the dataset...')
url_dataset = pd.read_csv('Machine-Learning-for-Security-Analysts/Malicious URLs.csv')
test_url = url_dataset['URLs'][4]
print('dataset has been installed')

loading the dataset...
dataset has been installed


In [ ]:
url_dataset.head()

,URLs,Class
0,freebase.com/view/en/bob_sirois,good
1,en.wikipedia.org/wiki/Joie_Lee,good
2,pipl.com/directory/people/Rejean/Beaudin,good
3,flickr.com/photos/teneyck/sets/72157610336209297/,good
4,ussoccer.com/News/Federation-Services/2009/06/...,good


import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_0, *['URLs'], **{})
chart

In [ ]:
url_dataset.shape

(420464, 2)

In [ ]:
#checking is there any missing values
url_dataset.isnull().sum()

URLs     0
Class    0
dtype: int64

In [ ]:
#counting no of good and bad url's
url_dataset['Class'].value_counts()

good    344821
bad      75643
Name: Class, dtype: int64

In [ ]:
#dataprocessing
x=url_dataset.drop(columns='Class',axis=1)
y=url_dataset['Class']

In [ ]:
# Perform Train/Test split


train_df, test_df = train_test_split(url_dataset, test_size=0.2, random_state=2)

labels = train_df['Class']
test_labels = test_df['Class']

print("\n Split Complete\n")


 Split Complete



In [ ]:
print(x)
print(y)

                                                     URLs
0                         freebase.com/view/en/bob_sirois
1                          en.wikipedia.org/wiki/Joie_Lee
2                pipl.com/directory/people/Rejean/Beaudin
3       flickr.com/photos/teneyck/sets/72157610336209297/
4       ussoccer.com/News/Federation-Services/2009/06/...
...                                                   ...
420459  ourorigins.org/genealogielistfirstname.aspx?an...
420460    simira.co.id/cifk/live.com/Account_Verified.htm
420461  kstatesports.com/sports/w-baskbl/spec-rel/ksu-...
420462  vh1.com/video/living-colour/9128/cult-of-perso...
420463     absoluteastronomy.com/topics/SummerSlam_(1990)

[420464 rows x 1 columns]
0         good
1         good
2         good
3         good
4         good
          ... 
420459    good
420460     bad
420461    good
420462    good
420463    good
Name: Class, Length: 420464, dtype: object


In [ ]:
def tokenizer(url):
  """Separates feature words from the raw data
  Keyword arguments:
    url ---- The full URL

  :Returns -- The tokenized words; returned as a list
  """

  # Split by slash (/) and dash (-)
  tokens = re.split('[/-]', url)

  for i in tokens:
    # Include the splits extensions and subdomains
    if i.find(".") >= 0:
      dot_split = i.split('.')

      # Remove .com and www. since they're too common
      if "com" in dot_split:
        dot_split.remove("com")
      if "www" in dot_split:
        dot_split.remove("www")

      tokens += dot_split

  return tokens

print("\nTokenizer defined\n")


Tokenizer defined



In [ ]:
print(test_url)
print('tokenized url is')
tokenized_url = tokenizer(test_url)
print(tokenized_url)
#print('toknaized url')
#print(tokenized_url)

ussoccer.com/News/Federation-Services/2009/06/University-Of-Miami-President-Donna-E-Shalala-Joins-Team-To-Bring-FIFA-World-Cup-To-United-States-In.aspx
tokenized url is
['ussoccer.com', 'News', 'Federation', 'Services', '2009', '06', 'University', 'Of', 'Miami', 'President', 'Donna', 'E', 'Shalala', 'Joins', 'Team', 'To', 'Bring', 'FIFA', 'World', 'Cup', 'To', 'United', 'States', 'In.aspx', 'ussoccer', 'In', 'aspx']


In [ ]:
for token in list(dict.fromkeys(tokenized_url)):
 print("{}  ->  {}".format(tokenized_url.count(token), token))



1  ->  ussoccer.com
1  ->  News
1  ->  Federation
1  ->  Services
1  ->  2009
1  ->  06
1  ->  University
1  ->  Of
1  ->  Miami
1  ->  President
1  ->  Donna
1  ->  E
1  ->  Shalala
1  ->  Joins
1  ->  Team
2  ->  To
1  ->  Bring
1  ->  FIFA
1  ->  World
1  ->  Cup
1  ->  United
1  ->  States
1  ->  In.aspx
1  ->  ussoccer
1  ->  In
1  ->  aspx


In [ ]:
c_vec = CountVectorizer(tokenizer = tokenizer)
c_out= c_vec.fit_transform([test_url])
print(c_out)

t_vec = TfidfVectorizer(tokenizer = tokenizer)
t_out= t_vec.fit_transform([test_url])
print(t_out)


  (0, 24)	1
  (0, 13)	1
  (0, 7)	1
  (0, 16)	1
  (0, 1)	1
  (0, 0)	1
  (0, 22)	1
  (0, 14)	1
  (0, 12)	1
  (0, 15)	1
  (0, 5)	1
  (0, 6)	1
  (0, 17)	1
  (0, 11)	1
  (0, 19)	1
  (0, 20)	2
  (0, 3)	1
  (0, 8)	1
  (0, 25)	1
  (0, 4)	1
  (0, 21)	1
  (0, 18)	1
  (0, 10)	1
  (0, 23)	1
  (0, 9)	1
  (0, 2)	1
  (0, 2)	0.18569533817705186
  (0, 9)	0.18569533817705186
  (0, 23)	0.18569533817705186
  (0, 10)	0.18569533817705186
  (0, 18)	0.18569533817705186
  (0, 21)	0.18569533817705186
  (0, 4)	0.18569533817705186
  (0, 25)	0.18569533817705186
  (0, 8)	0.18569533817705186
  (0, 3)	0.18569533817705186
  (0, 20)	0.3713906763541037
  (0, 19)	0.18569533817705186
  (0, 11)	0.18569533817705186
  (0, 17)	0.18569533817705186
  (0, 6)	0.18569533817705186
  (0, 5)	0.18569533817705186
  (0, 15)	0.18569533817705186
  (0, 12)	0.18569533817705186
  (0, 14)	0.18569533817705186
  (0, 22)	0.18569533817705186
  (0, 0)	0.18569533817705186
  (0, 1)	0.18569533817705186
  (0, 16)	0.18569533817705186
  (0, 7)	0.1856953

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
cvec = CountVectorizer(tokenizer = tokenizer)
train_count = cvec.fit_transform(train_df['URLs'])

tfidfvec = TfidfVectorizer(tokenizer = tokenizer)
train_tfidf = tfidfvec.fit_transform(train_df['URLs'])


In [ ]:
test_count = cvec.fit_transform(test_df['URLs'])
test_tfidf = tfidfvec.fit_transform(test_df['URLs'])


In [ ]:
model = MultinomialNB()



In [ ]:
print(labels)

model.fit(train_tfidf, labels)

model_train_prediction = model.predict(train_tfidf)
train_data_accuracy = accuracy_score(model_train_prediction,labels)
print(train_data_accuracy)




#precession and recall

271934    good
53429     good
369659    good
73727     good
338507     bad
          ... 
84434     good
95816     good
203245    good
100879    good
351400    good
Name: Class, Length: 336371, dtype: object
0.9849481673509309


In [ ]:
model.fit(test_tfidf,test_labels)
model_test_prediction = model.predict(test_tfidf)
test_accuracy_score = accuracy_score(model_test_prediction, test_labels)
print(test_accuracy_score)

0.9750752143460216


In [ ]:
count_model = MultinomialNB()
count_model.fit(train_count,labels)
model_train_prediction = count_model.predict(train_count)
model_train_accuracy = accuracy_score(model_train_prediction,labels)
print(model_train_accuracy)

0.9923507079980141


In [ ]:
count_model.fit(test_count,test_labels)
test_model_prediction = count_model.predict(test_count)
count_test_accuracy = accuracy_score(test_model_prediction,test_labels)
print(count_test_accuracy)

0.992710451523908


In [ ]:
logic_reg = LogisticRegression()
logic_reg.fit(train_tfidf,labels)
train_log_prediction = logic_reg.predict(train_tfidf)
train_logic_accuracy = accuracy_score(train_log_prediction,labels)
print(train_logic_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9714333280811961


In [ ]:
logic_reg.fit(test_tfidf,test_labels)
test_log_prediction = logic_reg.predict(test_tfidf)
test_logic_accuracy = accuracy_score(test_log_prediction,test_labels)
print(test_logic_accuracy)

0.9550973327149703


making predictive system

In [ ]:
new_input = ["br-icloud.com.br"]

new_input = tfidfvec.transform(new_input)
prediction = logic_reg.predict(new_input)
if(prediction == 'good'):
    print('url is benign')
else:
  print('url is malicious')

url is malicious


In [ ]:
new_input = ["qwedcrfc.com.br"]

new_input = tfidfvec.transform(new_input)
prediction = logic_reg.predict(new_input)
if(prediction == 'good'):
    print('url is benign')
else:
  print('url is malicious')

url is malicious


In [ ]:
new_input = ["google.com"]

new_input = tfidfvec.transform(new_input)
prediction = logic_reg.predict(new_input)
if(prediction == 'good'):
    print('url is benign')
else:
  print('url is malicious')

url is benign
